In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests
import pandas as pd
import string

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.get('https://www.idoc.idaho.gov/content/prisons/resident-client-search')

In [ ]:
combos = []
for letter1 in string.ascii_lowercase:
    for letter2 in string.ascii_lowercase:
        for letter3 in string.ascii_lowercase:
            combo = (letter1 + letter2 + letter3)
            combos.append(combo)
ids = []

In [ ]:
for combo in combos:
    driver.find_element('id', 'edit-last-name').send_keys(combo)
    driver.find_element('id', 'edit-submit').click()
    while True:
        try:
            next = driver.find_element('xpath', "//a[starts-with(@title, 'Go to next page')]")
        except:
            break
        links = driver.find_elements("xpath", "//a[starts-with(@href, '/content/prisons/resident-client-search/details/')]")
        for link in links:
            ids.append(link.text)
        next.click()
    driver.get('https://www.idoc.idaho.gov/content/prisons/resident-client-search')

In [ ]:
idaho = pd.DataFrame(columns=[])

In [ ]:
failed_ids = []

for id in ids:
    try:
        url = ('https://www.idoc.idaho.gov/content/prisons/resident-client-search/details/' + id)
        session = requests.Session()
        response = session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        details = soup.find('div', {'class': 'people-search-details-wrapper'}).find('div', {'class': 'col-md-3'}).find_all('div', {'class': 'well'})
        name_elements = details[0].find_all('strong')
        name = name_elements[0].text
        age = name_elements[3].next_sibling.text.strip(': ')
        status = name_elements[2].next_sibling.text.strip(': ')

        offenses = []
        counties = []
        sendates = []

        table = soup.find('table', {'class': 'table-ally table-design'}).find('tbody')

        offense_elements = table.find_all('div', text = 'Offense')
        county_elements = table.find_all('div', text = 'Sentencing County')
        sendate_elements = table.find_all('div', text = 'Sentence Satisfaction Date')

        fac = soup.find('strong', text = ['Mailing Address ', 'Supervising District ']).next_sibling.next_sibling.next_sibling.text

        for element in offense_elements:
            offense = element.next_sibling.text
            offenses.append(offense)

        for element in county_elements:
            county = element.next_sibling.text
            counties.append(county)
 
        for element in sendate_elements:
            sendate = element.next_sibling.text
            sendates.append(sendate)

        histories = [list(history) for history in zip(offenses, counties, sendates)]

        rows = []

        for history in histories:
            row = {
                'ID': id,
                'Name': name,
                'Age': age,
                'Facility': fac,
                'Offense Description': history[0],
                'Sentence Satisfaction Date': history[2],
                'County': history[1],
                'Status': status
                }
            rows.append(row)
            idaho = pd.concat([idaho, pd.DataFrame(rows)])

    except Exception as e:
        print(f"Failed to scrape {id}. Error: {e}")
        failed_ids.append(id)
        continue

In [ ]:
idaho.to_csv('/Users/tumendemberelshalkhaan/Desktop/felony_murder/raw_data/Idaho_Raw.csv', index=False)